In [1]:
import re
import string
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import newspaper
import glob

In [12]:
files = []
address = "C://"
round_cnt = 1
r_f_lnth = -1

while r_f_lnth != 0 and round_cnt <= 5:
    r_files = glob.glob(address)
    r_f_lnth = len(r_files)

    files.extend(r_files)
    address += "/*"

    round_cnt += 1 
len(files)

99110

In [13]:
def retrieve_docs_and_clean():
#     r = requests.get('https://news.ycombinator.com/')
#     soup = BeautifulSoup(r.content, 'html.parser')

#     link = []
#     for i in soup.find('div', {'class':'most__wrap'}).find_all('a'):
#       i['href'] = i['href'] + '?page=all'
#       link.append(i['href'])

    # Retrieve Paragraphs
#     documents = []
#     for i in link:
#       r = requests.get(i)
#       soup = BeautifulSoup(r.content, 'html.parser')

#       sen = []
#       for i in soup.find('div', {'class':'read__content'}).find_all('p'):
#           sen.append(i.text)
#       documents.append(' '.join(sen))

    # Clean Paragraphs
    documents_clean = []
    for d in files:
      document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
      document_test = re.sub(r'@\w+', '', document_test)
      document_test = document_test.lower()
      document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
      document_test = re.sub(r'[0-9]', '', document_test)
      document_test = re.sub(r'\s{2,}', ' ', document_test)
      documents_clean.append(document_test)

    return documents_clean

In [14]:
docs = retrieve_docs_and_clean()
docs

['c ',
 'c recycle bin',
 'c windows bt',
 'c windows ws',
 'c winreagent',
 'c aow drv log',
 'c apps',
 'c bootmgr',
 'c bootnxt',
 'c dicttobin',
 'c documents and settings',
 'c dumpstack log',
 'c dumpstack log tmp',
 'c esd',
 'c gog games',
 'c hiberfil sys',
 'c intel',
 'c local search',
 'c logs',
 'c onedrivetemp',
 'c pagefile sys',
 'c perflogs',
 'c program files',
 'c program files x ',
 'c programdata',
 'c recovery',
 'c riot games',
 'c swapfile sys',
 'c swsetup',
 'c system volume information',
 'c system sav',
 'c users',
 'c windows',
 'c recycle bin s ',
 'c recycle bin s ',
 'c recycle bin s ',
 'c recycle bin s ',
 'c windows bt dudownload',
 'c windows ws sources',
 'c winreagent scratch',
 'c apps obs',
 'c apps unity',
 'c dicttobin data txt',
 'c gog games the witcher wild hunt',
 'c intel gfxcplbatchfiles',
 'c intel inteloptanedata',
 'c local search files dictionary pkl',
 'c local search token dictionary pkl',
 'c logs player log',
 'c onedrivetemp s ',

In [15]:
# Create Term-Document Matrix with TF-IDF weighting
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)

# Create a DataFrame
df = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names())
print(df.head())
print(df.shape)

        0      1      2      3      4      5      6      7      8      9      \
aa        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
aaa       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
aaaa      0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
aaaaad    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
aaaaba    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

        ...  99100  99101  99102  99103  99104  99105  99106  99107  99108  \
aa      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
aaa     ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
aaaa    ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
aaaaad  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
aaaaba  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

        99109  
aa        0.0  
aaa       0.0  
aa

C:\Users\kantw\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [32]:
def get_similar_articles(q, df):
    print("query:", q)
    q = [q]
    q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
    
    sim = {}
    for i in range(1,1000):
        sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
    
    sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)

    for k, v in sim_sorted:
        if v != 0.0:
            print("Score:", v)
            print(docs[k])
            print()

In [33]:
get_similar_articles("log file", df)

query: log file
Score: 0.4542761519168108
c program files x bluestacks x api ms win core file l dll

Score: 0.4542761519168108
c program files x bluestacks x api ms win core file l dll

Score: 0.40215791466172035
c program files mozilla firefox api ms win core file l dll

Score: 0.40215791466172035
c program files mozilla firefox api ms win core file l dll

Score: 0.28345655492159816
c windows directx log

Score: 0.2659892401385401
c windows windowsupdate log

Score: 0.2365683278156361
c logs player log

Score: 0.2118887563035227
c windows comsetup log

Score: 0.2118887563035227
c windows setupact log

Score: 0.2118887563035227
c windows setuperr log

Score: 0.20197226241194677
c dumpstack log

Score: 0.19543991420656484
c windows dpinst log

Score: 0.19543991420656484
c windows dtcinstall log

Score: 0.19543991420656484
c windows pfro log

Score: 0.19543991420656484
c windows xhunters log

Score: 0.17417605155304208
c program files x steam gameoverlayrenderer log

Score: 0.17082719555